In [238]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [239]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [240]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [241]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [242]:
dim_fase = pd.read_sql_table('dim_fase_servicio', etl_conn) 
estado_servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)
fecha = pd.read_sql_table('dim_fecha', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)

In [243]:
estado_servicio

,id,fecha,hora,foto,observaciones,estado_id,servicio_id,es_prueba,foto_binary
0,1014,2024-01-29,01:13:32,foto,4 tubos,4,226,False,None
1,1484,2024-01-30,18:45:12,foto,Demora,3,79,True,None
2,2829,2024-02-06,11:34:04,foto,Compra exitosa,5,613,False,None
3,1888,2024-02-01,14:50:39,foto,Zzxzz,4,376,False,None
4,32312,2024-04-06,16:11:21,foto,No,3,7164,True,None
...,...,...,...,...,...,...,...,...,...
128397,128805,2024-08-31,15:01:03,foto,Con mensajero Asignado,2,28467,True,None
128398,128806,2024-08-31,15:03:42,foto,Con mensajero Asignado,2,28466,True,None
128399,128807,2024-08-31,15:04:26,foto,Recojo,4,28466,False,None
128400,128808,2024-08-31,15:13:47,foto,Este servicio rs moto csrguero,3,28467,True,None


In [244]:
print(len(estado_servicio))

128402


In [245]:
estado_servicio.drop(columns=["foto","observaciones","es_prueba","foto_binary"], inplace=True)
estado_servicio

,id,fecha,hora,estado_id,servicio_id
0,1014,2024-01-29,01:13:32,4,226
1,1484,2024-01-30,18:45:12,3,79
2,2829,2024-02-06,11:34:04,5,613
3,1888,2024-02-01,14:50:39,4,376
4,32312,2024-04-06,16:11:21,3,7164
...,...,...,...,...,...
128397,128805,2024-08-31,15:01:03,2,28467
128398,128806,2024-08-31,15:03:42,2,28466
128399,128807,2024-08-31,15:04:26,4,28466
128400,128808,2024-08-31,15:13:47,3,28467


## Transformation

### Hour Process

In [246]:
hora_to_process = estado_servicio[['id','hora']]
hora_to_process['hora'] = hora_to_process['hora'].astype(str)
# Elimina los milisegundos dividiendo en el punto y tomando solo la parte de HH:MM:SS
hora_to_process['hora'] = hora_to_process['hora'].str.split('.').str[0]
# Extrae solo la hora como un número entero
hora_to_process['hora_integer'] = pd.to_datetime(hora_to_process['hora'], format='%H:%M:%S').dt.hour
estado_servicio = pd.merge(estado_servicio, hora_to_process[["hora_integer", "id"]], left_on="id", right_on="id", how="left")

estado_servicio = pd.merge(estado_servicio, hora[["key_hora"]], left_on="hora_integer", right_on="key_hora", how="left")
estado_servicio.head(10)


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1289646949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hora_to_process['hora'] = hora_to_process['hora'].astype(str)
C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1289646949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hora_to_process['hora'] = hora_to_process['hora'].str.split('.').str[0]
C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1289646949.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,id,fecha,hora,estado_id,servicio_id,hora_integer,key_hora
0,1014,2024-01-29,01:13:32,4,226,1,1
1,1484,2024-01-30,18:45:12,3,79,18,18
2,2829,2024-02-06,11:34:04,5,613,11,11
3,1888,2024-02-01,14:50:39,4,376,14,14
4,32312,2024-04-06,16:11:21,3,7164,16,16
5,2426,2024-02-04,11:15:40,3,377,11,11
6,48803,2024-05-03,06:11:39,3,10910,6,6
7,3323,2024-02-07,16:26:03,2,746,16,16
8,3886,2024-02-09,13:25:05,6,842,13,13
9,4211,2024-02-10,13:46:43,2,930,13,13


### Date Process

In [247]:
estado_servicio = pd.merge(estado_servicio, fecha[["date", "key_fecha"]], left_on="fecha", right_on="date", how="left")
estado_servicio

,id,fecha,hora,estado_id,servicio_id,hora_integer,key_hora,date,key_fecha
0,1014,2024-01-29,01:13:32,4,226,1,1,2024-01-29,385
1,1484,2024-01-30,18:45:12,3,79,18,18,2024-01-30,386
2,2829,2024-02-06,11:34:04,5,613,11,11,2024-02-06,393
3,1888,2024-02-01,14:50:39,4,376,14,14,2024-02-01,388
4,32312,2024-04-06,16:11:21,3,7164,16,16,2024-04-06,453
...,...,...,...,...,...,...,...,...,...
128397,128805,2024-08-31,15:01:03,2,28467,15,15,2024-08-31,600
128398,128806,2024-08-31,15:03:42,2,28466,15,15,2024-08-31,600
128399,128807,2024-08-31,15:04:26,4,28466,15,15,2024-08-31,600
128400,128808,2024-08-31,15:13:47,3,28467,15,15,2024-08-31,600


### Phase Process

In [248]:
dim_fase.rename(columns={'id':'dim_estado_id'}, inplace=True)
estado_servicio = pd.merge(estado_servicio, dim_fase[["dim_estado_id", "key_fase_servicio"]], left_on="estado_id", right_on="dim_estado_id", how="left")
estado_servicio

,id,fecha,hora,estado_id,servicio_id,hora_integer,key_hora,date,key_fecha,dim_estado_id,key_fase_servicio
0,1014,2024-01-29,01:13:32,4,226,1,1,2024-01-29,385,4,0
1,1484,2024-01-30,18:45:12,3,79,18,18,2024-01-30,386,3,2
2,2829,2024-02-06,11:34:04,5,613,11,11,2024-02-06,393,5,1
3,1888,2024-02-01,14:50:39,4,376,14,14,2024-02-01,388,4,0
4,32312,2024-04-06,16:11:21,3,7164,16,16,2024-04-06,453,3,2
...,...,...,...,...,...,...,...,...,...,...,...
128397,128805,2024-08-31,15:01:03,2,28467,15,15,2024-08-31,600,2,5
128398,128806,2024-08-31,15:03:42,2,28466,15,15,2024-08-31,600,2,5
128399,128807,2024-08-31,15:04:26,4,28466,15,15,2024-08-31,600,4,0
128400,128808,2024-08-31,15:13:47,3,28467,15,15,2024-08-31,600,3,2


## calculo tiempo por fase_servicio

In [249]:
# Ordenar por 'servicio_id' y 'fecha'
estado_servicio = estado_servicio.sort_values(by=['servicio_id', 'fecha'])

# Asegurarse de que la columna 'hora' esté en formato timedelta
estado_servicio['hora'] = pd.to_timedelta(estado_servicio['hora'].astype(str))

# Calcular la diferencia en días entre fases consecutivas dentro de cada servicio
estado_servicio['dias_de_demora'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: x.diff().dt.days)

# asignar el resultado a la columna directamente para evitar errores jeje
estado_servicio['dias_de_demora'] = estado_servicio['dias_de_demora'].fillna(0)

# Calcular la diferencia en horas entre fases consecutivas dentro de cada servicio
estado_servicio['hora_de_demora'] = estado_servicio.groupby('servicio_id')['hora'].transform(lambda x: x.diff())

# Corregir las horas negativas: Si la hora actual es menor que la anterior, agregar 24 horas
estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: x if x >= pd.Timedelta(0) else x + pd.Timedelta(days=1))

# Convertir la diferencia en horas
estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# Lo mismo para la columna 'hora_de_demora'
estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].fillna(0)

# Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
estado_servicio['hora'] = estado_servicio['hora'].dt.total_seconds() / 3600
estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: round(x, 2))  # Redondear a 2 decimales


estado_servicio.head(20)


,id,fecha,hora,estado_id,servicio_id,hora_integer,key_hora,date,key_fecha,dim_estado_id,key_fase_servicio,dias_de_demora,hora_de_demora
301,6,2023-09-19,16.371667,1,7,16,16,2023-09-19,253,1,4,0.0,0.00
667,117,2023-10-13,17.855556,2,7,17,17,2023-10-13,277,2,5,24.0,1.48
686,134,2023-10-31,12.046901,4,7,12,12,2023-10-31,295,4,0,18.0,18.19
687,135,2023-10-31,17.131944,5,7,17,17,2023-10-31,295,5,1,0.0,5.09
688,136,2023-10-31,12.266667,6,7,12,12,2023-10-31,295,6,3,0.0,19.13
302,7,2023-09-19,16.501389,1,8,16,16,2023-09-19,253,1,4,0.0,0.00
792,241,2023-12-20,20.245278,2,8,20,20,2023-12-20,345,2,5,92.0,3.74
5902,5355,2024-02-14,15.571667,4,8,15,15,2024-02-14,401,4,0,56.0,19.33
34483,34061,2024-04-09,16.143056,5,8,16,16,2024-04-09,456,5,1,55.0,0.57
304,8,2023-09-19,16.501389,1,9,16,16,2023-09-19,253,1,4,0.0,0.00


## Calculo atributo tiempo promedio general

In [250]:
# Convertir 'hora_de_demora' a timedelta si aún no está en el formato adecuado
estado_servicio['hora_de_demora'] = pd.to_timedelta(estado_servicio['hora_de_demora'], unit='h')

# Calcular el tiempo total en días considerando solo las fechas, con dos decimales
estado_servicio['tiempo_total'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: (x.max() - x.min()).days).round(2)

# Calcular el tiempo total en horas sumando las horas de demora de cada fase por servicio
estado_servicio['tiempo_total_horas'] = estado_servicio.groupby('servicio_id')['hora_de_demora'].transform('sum').dt.total_seconds() / 3600
estado_servicio['tiempo_total_horas'] = estado_servicio['tiempo_total_horas'].round(2)

# Calcular el número de fases por servicio
estado_servicio['numero_fases'] = estado_servicio.groupby('servicio_id')['fecha'].transform('count')

# Calcular el tiempo promedio por fase en días (dividiendo el tiempo total de días entre el número de fases)
estado_servicio['tiempo_promedio_dias'] = (estado_servicio['tiempo_total'] / estado_servicio['numero_fases']).round(2)

# Calcular el tiempo promedio en horas por fase con dos decimales (dividiendo el tiempo total de horas entre el número de fases)
estado_servicio['tiempo_promedio_horas'] = (estado_servicio['tiempo_total_horas'] / estado_servicio['numero_fases']).round(2)

# Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# Mostrar los primeros 10 resultados
estado_servicio.head(10) 


,id,fecha,hora,estado_id,servicio_id,hora_integer,key_hora,date,key_fecha,dim_estado_id,key_fase_servicio,dias_de_demora,hora_de_demora,tiempo_total,tiempo_total_horas,numero_fases,tiempo_promedio_dias,tiempo_promedio_horas
301,6,2023-09-19,16.371667,1,7,16,16,2023-09-19,253,1,4,0.0,0.00,42,43.89,5,8.40,8.78
667,117,2023-10-13,17.855556,2,7,17,17,2023-10-13,277,2,5,24.0,1.48,42,43.89,5,8.40,8.78
686,134,2023-10-31,12.046901,4,7,12,12,2023-10-31,295,4,0,18.0,18.19,42,43.89,5,8.40,8.78
687,135,2023-10-31,17.131944,5,7,17,17,2023-10-31,295,5,1,0.0,5.09,42,43.89,5,8.40,8.78
688,136,2023-10-31,12.266667,6,7,12,12,2023-10-31,295,6,3,0.0,19.13,42,43.89,5,8.40,8.78
302,7,2023-09-19,16.501389,1,8,16,16,2023-09-19,253,1,4,0.0,0.00,203,23.64,4,50.75,5.91
792,241,2023-12-20,20.245278,2,8,20,20,2023-12-20,345,2,5,92.0,3.74,203,23.64,4,50.75,5.91
5902,5355,2024-02-14,15.571667,4,8,15,15,2024-02-14,401,4,0,56.0,19.33,203,23.64,4,50.75,5.91
34483,34061,2024-04-09,16.143056,5,8,16,16,2024-04-09,456,5,1,55.0,0.57,203,23.64,4,50.75,5.91
304,8,2023-09-19,16.501389,1,9,16,16,2023-09-19,253,1,4,0.0,0.00,100,3.05,2,50.00,1.52


## Eliminar columnas no necesarias

In [251]:
estado_servicio.drop(columns=['id','fecha','hora','estado_id','hora_integer','date','dim_estado_id'], inplace=True)
estado_servicio.head(10)

,servicio_id,key_hora,key_fecha,key_fase_servicio,dias_de_demora,hora_de_demora,tiempo_total,tiempo_total_horas,numero_fases,tiempo_promedio_dias,tiempo_promedio_horas
301,7,16,253,4,0.0,0.00,42,43.89,5,8.40,8.78
667,7,17,277,5,24.0,1.48,42,43.89,5,8.40,8.78
686,7,12,295,0,18.0,18.19,42,43.89,5,8.40,8.78
687,7,17,295,1,0.0,5.09,42,43.89,5,8.40,8.78
688,7,12,295,3,0.0,19.13,42,43.89,5,8.40,8.78
302,8,16,253,4,0.0,0.00,203,23.64,4,50.75,5.91
792,8,20,345,5,92.0,3.74,203,23.64,4,50.75,5.91
5902,8,15,401,0,56.0,19.33,203,23.64,4,50.75,5.91
34483,8,16,456,1,55.0,0.57,203,23.64,4,50.75,5.91
304,9,16,253,4,0.0,0.00,100,3.05,2,50.00,1.52


In [252]:
len(estado_servicio)

128402

## Load

In [253]:
estado_servicio.to_sql("hecho_servicio_fase", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

402